In [1]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 878, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 878 (delta 68), reused 100 (delta 64), pack-reused 768
Receiving objects: 100% (878/878), 62.91 MiB | 8.72 MiB/s, done.
Resolving deltas: 100% (573/573), done.
Updating files: 100% (413/413), done.


Características

Valor nominal: Depende emisora, $100$ MXN (Bonos M), $1,000$ USD (United Mexican States: UMS).

Plazo: Depende emisora, 3, 5, 7, 10, 20 y 30 años (Bonos M), y 5, 10, 15, 20, 25 y 30 años (UMS).

Colocación: Subasta de Banco de México.

Negociación: Depende emisora, a tasa de rendimiento (Bonos M), a tasa de rendimiento y en USD (UMS)

Intereses: Depende de emisora, 182 días (Bonos M), 6 meses o anual (la tasa puede ser revisable). 


In [2]:

import pandas as pd
import numpy as np
import math 

btasadesc="RiesgosFinancieros/2020-1/Insumos/tasa_yield.txt"
tfcupon=[0.065,	0.0675,	0.07,	0.075,	0.078] #Tasafija del cupón
plazos=[378, 405,	550,	1200,	1800] #Vencimiento del bono
plazocupon=[182,	182, 182, 182, 182] #plazos fijos de cada cupón
contratos=[22000, -29000, 29000, -46000, 10000] #posición invertida
nominal=100
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada

In [3]:
#carga de tasas de descuento
data1=pd.read_csv(btasadesc, delimiter="\t")
n=data1.shape[0]
m_orig=data1.shape[1]
X_orig=data1.iloc[1:n,1:m_orig]
nodos=list(data1.columns[1:])
nodos = data1.columns[1:].astype(int)

In [ ]:
#Tasa alambrada

def talamb(nodos, curva,plazos):
  n= max(len(plazos),1)
  m = max(len(nodos),1)
  TC = np.matrix(np.zeros((n,1)))
  TL = np.matrix(np.zeros((n,1)))
  TF = np.matrix(np.zeros((n,1)))

  for j in range(1,n+1):
    i = 0
    while True:
      if nodos[i] <= plazos[j] <= nodos[i+1]:
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j,1]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))**((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      elif plazos[j] < nodos[0]:
        TC[j]=curva[0]
        TL[j]=curva[0]
        TF[j]=curva[0]
        break
      elif plazos[j]>nodos[m]:
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
      else:
        i+=1
    return np.matrix(TF.append()).T


In [12]:
from scipy.interpolate import interp1d
m=len(plazos)
X=np.matrix(np.zeros((n-1,m)))
for i in range(0,n-1):
  if itpl==0:    
    y_interp = interp1d(nodos,X_orig.iloc[i,:])
    X[[i]] = y_interp(plazos)
    # Problemas con TALAMB
  else:
    talamb(nodos,X_orig[i,],plazos)

$$V_{ty}=\sum_{i=1}^{n}\frac{N\cdot C \cdot t_{c}\cdot p_c/360}{(1+t_{n} \cdot p_c/360)^{(p_i/p_c)}} + \frac{N\cdot C}{(1+t_{n} \cdot p_c/360)^{(p_n/p_c)}} \quad \textrm{ó}$$
	$$ V_{ty}=\Bigg(\frac{N\cdot C \cdot t_{c}\cdot p_c}{360}\bigg( \frac{(1-(1+\frac{t_n\cdot p_c}{360})^{-(\Big[\frac{p_n}{p_c}\Big]+1)})}{t_n\cdot p_c/360}\bigg) + \frac{N\cdot C}{(1+\frac{t_{n} \cdot p_c}{360})^{(\Big[\frac{p_n}{p_c}\Big]+1)}}\Bigg)\bigg(1+\frac{t_n\cdot p_c}{360}\bigg)^{1-\frac{p_1}{p_c}}$$			
	
Donde $V_{ty}$: Valor del bono con tasa yield $t_n$ (Precio Sucio)
$t_n$: tasa de interés al vencimiento.
$N$: Valor Nominal del bono
$C$: Número de contratos
$p_{c}$: Plazo fijo para cada pago de intereses del cupón
$p_{i}$: Plazo acumulado (en días) al cupón $i$
$t_{c}$: Tasa cupón fija. 
En este caso se tiene un factor de riesgo subyacente y UN SÓLO factor de riesgo que son los cupones. 


In [27]:
def bonoMyield(x, plazos, plazocupon, tfcupon, nominal, contratos):
  x=x0
  N = []
  a = []
  p1 = []
  resultado = []
  for i in range(0,len(plazos)):
    N += [int(plazos[i]/plazocupon[i]+1)]
    a += [(1-(1+x[0,i]*plazocupon[i]/360)**(-N[i]))/(plazocupon[i]*x[0,i]/360)]
    p1 += [plazos[i]-plazocupon[i]*(N[i]-1)]
    resultado += [(contratos[i]*nominal*tfcupon[i]*plazocupon[i]/360)*a[i]+(contratos[i]*nominal)/((1+x[0,i]*plazocupon[i]/360)**N[i])*(1+x[0,i]*plazocupon[i]/360)**(1-p1[i]/plazocupon[i])]
  return resultado

x0=X[0,:]
bonoMyield(x0,plazos, plazocupon, tfcupon, nominal, contratos)

[2228469.672179762,
 -2932833.500133448,
 2953854.3398862453,
 -4627183.800148209,
 1008099.3358986827]

In [40]:
vp = []
plazocupcurr = []
for j in range(0,m):
  vp += [np.matrix(np.zeros(N[j]))]
  for i in range(0,N[j]):
    if i==N[j]:
  	  plazocupcurr += [plazini[j]+plazocupon[j]*(i)]
      #ME QUEDE AQUÍ, REVISAR EL FINAL
      # vp[i] = (contratos[j]*nominal*tfcupon[j]*plazocupon[j]/360+contratos[j]*nominal)*(1+x[0,j]*plazocupon[j]/360)**(-(plazocupcurr/plazocupon[j]))
      # svp[j] = vp[i]+svp[j]  

NameError: ignored

In [41]:
def bonotasafija(x, plazos, plazocupon, tfcupon, nominal, contratos):
  m=max(len(plazos),1)
  N=np.matrix(np.zeros((1,m)))
  N = []
  plazini = []
  for i in range(0,m):
    N += [int(plazos[i]/plazocupon[i])+1]
    plazini += [plazos[i]-plazocupon[i]*(N[i]-1)]
    svp = np.matrix(np.zeros((1,m)))
###########
  for j in range(0,m):
    vp=np.matrix(np.zeros(N[j]))
    for i in range(0,N[j]):
      if i==N[j]:
 			  plazocupcurr=plazini[j]+plazocupon[j]*(i-1)
			  vp[i]=(contratos[j]*nominal*tfcupon[j]*plazocupon[j]/360+contratos[j]*nominal)*(1+x[j]*plazocupon[j]/360)**(-(plazocupcurr/plazocupon[j]))
			  svp[j]=vp[i]+svp[j]       
      else:
 			  plazocupcurr=plazini[j]+plazocupon[j]*(i-1)
			  vp[i]=contratos[j]*nominal*tfcupon[j]*plazocupon[j]/360*(1+x[j]*plazocupon[j]/360)**(-(plazocupcurr/plazocupon[j]))
			  svp[j]=vp[i]+svp[j]
  return svp
V0=bonotasafija(x0, plazos, plazocupon, tfcupon, nominal, contratos) #Valor del portafolio
V0


TabError: ignored

In [ ]:


		svp
}


V0=bonotasafija(x0, plazos, plazocupon, tfcupon, nominal, contratos) #Valor del portafolio
V0

In [ ]:
def 